In [1]:
import pandas as pd
import os
import glob

inputPath = './data/records_samples/'

# Load dataframe with records
Create records_samples folder in data if not there
Place records.pkl and samples.pkl in folder from shared folder in google drive

In [ ]:
df_record = pd.read_pickle(os.path.join(inputPath,'records.pkl'))
df_record.head()

# Load DNorm input data
First part of notebook: we will create the input data for dnorm 
From a pandas dataframe that contains the records (records.pkl)
Input whether you want all records or a certain number of records 
 - For all records: all_rows=1
 - For fixed number, all_rows=0 and n_rows=[number of rows you want]
 
This will output DNorm input data to data/DNorm_input.txt
 
To run DNorm you then place this into the DNorm folder and run code 
For DNorm instructions download java app here: ncbi.nlm.nih.gov/research/bionlp/Tools/dnorm and look at readme text file inside folder)

In [ ]:
all_rows = 1
n_rows = 1000


if all_rows==1:
    n_rows = len(df_record)

rec4input = df_record.loc[:n_rows,:]

outF = open(os.path.join(inputPath, 'dnorm_input.txt'), "w")
# build list of strings
rec4input = rec4input.reset_index()
for i in range(len(rec4input)):
    Id = rec4input.loc[i,:].Id
    title = rec4input.loc[i,:].title
    summary = rec4input.loc[i,:].summary
    outF.write(Id)
    outF.write('\t')
    outF.write(title +'. '+ summary)
    outF.write("\n")

# Turn DNorm output into pandas df
 - After running DNorm, take text output file and paste into output_data_path folder
 - Turn into dataframe and save to output path

In [ ]:
output_data_path = './data/DNorm/'

if os.path.isdir(output_data_path)!=1:
    os.mkdir(output_data_path)

output_file_path = os.path.join(output_data_path, 'record_tags.txt')

with open(output_file_path,'r') as f:
    data = f.read().splitlines()
    
table = [d.split('\t') for d in data]
headers = ['Id', 'start', 'end', 'disease_tag', 'ONTid']
df = pd.DataFrame(table, columns=headers)

s = df.ONTid.str.split(':')
df_aux = pd.DataFrame.from_items(zip(s.index, s.values)).T
df_aux.columns = ['ont', 'unique_id']
df['ont'] = df_aux['ont']
df['unique_id'] = df_aux['unique_id']
df = df.drop('ONTid', axis = 1)

df.to_pickle(os.path.join(output_data_path,'disease_tags.pkl'))